### ***8***

# LRT for Binary Classification 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%reset-f

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy
path = '/content/drive/MyDrive/ML/data_q4_q5.xlsx' 
df = pd.read_excel(path)
insts = df.to_numpy()
m= len(insts[:,0]) #No of instinces
for i in range(m):
  insts[i,-1] = (ord(insts[i,-1])-66)/11
n = len(insts[0,:])-1 #no of features 
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std()
np.random.shuffle(insts)

In [4]:
def cal_prior(insts,c): #insts of training set , no of classes
  p= np.ones(c)
  l = len(insts[:,-1])
  for i in range(c):
    p[i] = np.count_nonzero(insts[:,-1] == i)/l
  return p # returns prior probabilities matrix

def cal_mean(insts,m,n): 
  mean = np.ones(n)
  for i in range(n):
    mean[i] = np.sum(insts[:,i])/m
  return mean
def confmat(y_pred,y_ts): #CONFUTION MATRIX
  a, b, c, d = 0, 0, 0, 0
  for i in range(len(y_ts)):
      if y_ts[i] == 0:
          if y_pred[i] == 0:
              a = a + 1
          if y_pred[i] == 1:
              b = b + 1
      if y_ts[i] == 1:
          if y_pred[i] == 0:
              c = c + 1
          if y_pred[i] == 1:
              d = d + 1
  return a, b, c, d

In [5]:
K = 5 # no of folds
avg_accuracy = 0
avg_sensitivity = 0
avg_specificity = 0
for k in range(K):
  insts_te = insts[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  m_tr = len(insts_tr[:,0])
  m_te = len(insts_te[:,0])
  P_y = cal_prior(insts_tr,2) #prior probability vector
   #mean vector
  insts_y1 = insts_tr[insts_tr[:,-1]==0] #insts for class 0 in training set
  insts_y2 = insts_tr[insts_tr[:,-1]==1] #insts for class 1 in training set
  mean1 = cal_mean(insts_y1,len(insts_y1[:,0]),n) #mean vector for class 0
  mean2 = cal_mean(insts_y2,len(insts_y2[:,0]),n) #mean vector for class 1
  cov1  = np.cov((insts_y1[:,0:n].astype(np.float32)).T)
  cov2  = np.cov((insts_y2[:,0:n].astype(np.float32)).T)
  coeff1 = 1/(((2*3.14)**(n/2))*np.linalg.det(cov1)**0.5)
  coeff2 = 1/(((2*3.14)**(n/2))*np.linalg.det(cov2)**0.5)
  y_pred = []
  for i in range(m_te):
    l1 = coeff1*np.exp(-0.5*np.dot(np.dot((insts_te[i,0:n] - mean1),np.linalg.inv(cov1)),(insts_te[i,0:n] - mean1).T))
    l2 = coeff2*np.exp(-0.5*np.dot(np.dot((insts_te[i,0:n] - mean2),np.linalg.inv(cov2)),(insts_te[i,0:n] - mean2).T))
    if (l1/P_y[1]) > (l2/P_y[0]):
      y_pred.append(0)
    else:
      y_pred.append(1)
  y_acu = insts_te[:,-1]
  a, b, c, d = confmat(y_pred,y_acu)
  avg_accuracy += (a+d)/(a+b+c+d)
  avg_sensitivity += (a)/(a+b)
  avg_specificity += (d)/(d+c)
print('accuracy: ',avg_accuracy/K,'\nsensitivity: ',avg_sensitivity/K,'\nspecificity: ',avg_specificity/K)

accuracy:  0.9578481602235678 
sensitivity:  0.9633122262989703 
specificity:  0.9517830478700045
